In [5]:
import pandas as pd

In [11]:
# ! pip freeze

## Load data

In [94]:
df=pd.read_excel('data/raw/tips_data.xlsx')

In [95]:
df

,name,activity,timestamp,money_spent
0,Jhon Doe,beach,2017-08-04 12:00:00,50
1,Joey Tribbiani,football,2017-08-04 08:00:00,60
2,Jhon Doe,beach,2017-08-04 07:00:00,30
3,Harry Kane,party,2017-08-04 07:00:00,20
4,Phoebe Buffay,travel,2017-08-04 10:00:00,40
5,Phoebe Buffay,football,2017-08-04 14:00:00,50
6,Jhon Doe,beach,2017-08-04 08:00:00,60
7,Phoebe Buffay,party,2017-08-04 16:00:00,70
8,Phoebe Buffay,football,2017-08-04 18:00:00,50


## 1. String commands

In [96]:
df['name'] = df.name.str.split(' ', expand=True)

In [97]:
df

,name,activity,timestamp,money_spent
0,Jhon,beach,2017-08-04 12:00:00,50
1,Joey,football,2017-08-04 08:00:00,60
2,Jhon,beach,2017-08-04 07:00:00,30
3,Harry,party,2017-08-04 07:00:00,20
4,Phoebe,travel,2017-08-04 10:00:00,40
5,Phoebe,football,2017-08-04 14:00:00,50
6,Jhon,beach,2017-08-04 08:00:00,60
7,Phoebe,party,2017-08-04 16:00:00,70
8,Phoebe,football,2017-08-04 18:00:00,50


In [98]:
 df.name.str.split(' ', expand=True)

,0
0,Jhon
1,Joey
2,Jhon
3,Harry
4,Phoebe
5,Phoebe
6,Jhon
7,Phoebe
8,Phoebe


## 2. Group by and value_counts

In [99]:
df.groupby('name')['activity'].value_counts()

name    activity
Harry   party       1
Jhon    beach       3
Joey    football    1
Phoebe  football    2
        party       1
        travel      1
Name: activity, dtype: int64

## 3. Unstack

By doing unstack we are transforming the last level of the index to the columns. All the activities values will now be the columns of a the dataframe and when a person has not done a certain activity this feature will get Nan value. Fillna fills all these missing values (activities which were not visited by the person) with 0.

In [100]:
df.groupby('name')['activity'].value_counts().unstack().fillna(0)

activity,beach,football,party,travel
name,,,,
Harry,0.0,0.0,1.0,0.0
Jhon,3.0,0.0,0.0,0.0
Joey,0.0,1.0,0.0,0.0
Phoebe,0.0,2.0,1.0,1.0


## 3. groupby, diff, shift, and loc + A great tip for efficiency

Knowing the time differences between person activities can be quite interesting for predicting who is the most fun person. How long did a person hang out in a party? how long did he/she hang out at the the beach? This might be useful for us as a feature, depends on the activity.
The most straight forward way to calculate the time differences would be to groupby the person name and them calculate the difference on the timestamp field using diff():

In [101]:
df.groupby('name').head()

,name,activity,timestamp,money_spent
0,Jhon,beach,2017-08-04 12:00:00,50
1,Joey,football,2017-08-04 08:00:00,60
2,Jhon,beach,2017-08-04 07:00:00,30
3,Harry,party,2017-08-04 07:00:00,20
4,Phoebe,travel,2017-08-04 10:00:00,40
5,Phoebe,football,2017-08-04 14:00:00,50
6,Jhon,beach,2017-08-04 08:00:00,60
7,Phoebe,party,2017-08-04 16:00:00,70
8,Phoebe,football,2017-08-04 18:00:00,50


In [102]:
df = df.sort_values(by=['name','timestamp'])
df['time_diff'] = df.groupby('name')['timestamp'].diff()

In [103]:
df

,name,activity,timestamp,money_spent,time_diff
3,Harry,party,2017-08-04 07:00:00,20,NaT
2,Jhon,beach,2017-08-04 07:00:00,30,NaT
6,Jhon,beach,2017-08-04 08:00:00,60,01:00:00
0,Jhon,beach,2017-08-04 12:00:00,50,04:00:00
1,Joey,football,2017-08-04 08:00:00,60,NaT
4,Phoebe,travel,2017-08-04 10:00:00,40,NaT
5,Phoebe,football,2017-08-04 14:00:00,50,04:00:00
7,Phoebe,party,2017-08-04 16:00:00,70,02:00:00
8,Phoebe,football,2017-08-04 18:00:00,50,02:00:00


If you have a lot of data and you want to save some time (this can be about 10 times faster depends on your data size) you can skip the groupby and just do the diff after sorting the data and then deleting the first row of each person which is not relevant.

In [104]:
df = df.sort_values(by=['name','timestamp'])
df['time_diff'] = df.groupby('name')['timestamp'].diff()
# df.loc[df.name != df.name.shift(), 'time_diff'] = None

In [105]:
df

,name,activity,timestamp,money_spent,time_diff
3,Harry,party,2017-08-04 07:00:00,20,NaT
2,Jhon,beach,2017-08-04 07:00:00,30,NaT
6,Jhon,beach,2017-08-04 08:00:00,60,01:00:00
0,Jhon,beach,2017-08-04 12:00:00,50,04:00:00
1,Joey,football,2017-08-04 08:00:00,60,NaT
4,Phoebe,travel,2017-08-04 10:00:00,40,NaT
5,Phoebe,football,2017-08-04 14:00:00,50,04:00:00
7,Phoebe,party,2017-08-04 16:00:00,70,02:00:00
8,Phoebe,football,2017-08-04 18:00:00,50,02:00:00


BTW — the useful .Shift command shift all the column down per one space, so we can see on which row this column is changing by doing this: df.name!=df.name.shift().
And .loc command is the most recommended way to set values for a column for specific indices.
To change the time_diff to seconds units:

In [106]:
# df['time_diff'] = df.time_diff.dt.total_seconds()

To get the duration per row:

In [107]:
df['row_duration'] = df.time_diff.shift(-1)

In [108]:
df

,name,activity,timestamp,money_spent,time_diff,row_duration
3,Harry,party,2017-08-04 07:00:00,20,NaT,NaT
2,Jhon,beach,2017-08-04 07:00:00,30,NaT,01:00:00
6,Jhon,beach,2017-08-04 08:00:00,60,01:00:00,04:00:00
0,Jhon,beach,2017-08-04 12:00:00,50,04:00:00,NaT
1,Joey,football,2017-08-04 08:00:00,60,NaT,NaT
4,Phoebe,travel,2017-08-04 10:00:00,40,NaT,04:00:00
5,Phoebe,football,2017-08-04 14:00:00,50,04:00:00,02:00:00
7,Phoebe,party,2017-08-04 16:00:00,70,02:00:00,02:00:00
8,Phoebe,football,2017-08-04 18:00:00,50,02:00:00,NaT


## 4. Cumcount and Cumsum

This are two really cool Ufuncs which can help you with many things. Cumcount create a cumulative count. For example we can take only the second activity for each person by grouping by the person name and then applying cumcount. This will just give a count for the activities by their order. Than we can take only the second activity for each person by doing ==1 (or the third activity by doing ==2) and applying the indices on the original sorted dataframe.

In [109]:
df = df.sort_values(by=['name','timestamp'])
df2 = df[df.groupby('name').cumcount()==1]
df2

,name,activity,timestamp,money_spent,time_diff,row_duration
6,Jhon,beach,2017-08-04 08:00:00,60,01:00:00,04:00:00
5,Phoebe,football,2017-08-04 14:00:00,50,04:00:00,02:00:00


In [110]:
df = df.sort_values(by=['name','timestamp'])
df2 = df[df.groupby('name').cumcount()==2]
df2

,name,activity,timestamp,money_spent,time_diff,row_duration
0,Jhon,beach,2017-08-04 12:00:00,50,04:00:00,NaT
7,Phoebe,party,2017-08-04 16:00:00,70,02:00:00,02:00:00


Cumsum is just a cummulative summary of a numeric cell. For example you can add the money the person spend in each activity as an additional cell and then summarize the money spent by a person at each time of the day using:

In [111]:
df = df.sort_values(by=['name','timestamp'])
df['money_spent_so_far'] = df.groupby('name')['money_spent'].cumsum()
df

,name,activity,timestamp,money_spent,time_diff,row_duration,money_spent_so_far
3,Harry,party,2017-08-04 07:00:00,20,NaT,NaT,20
2,Jhon,beach,2017-08-04 07:00:00,30,NaT,01:00:00,30
6,Jhon,beach,2017-08-04 08:00:00,60,01:00:00,04:00:00,90
0,Jhon,beach,2017-08-04 12:00:00,50,04:00:00,NaT,140
1,Joey,football,2017-08-04 08:00:00,60,NaT,NaT,60
4,Phoebe,travel,2017-08-04 10:00:00,40,NaT,04:00:00,40
5,Phoebe,football,2017-08-04 14:00:00,50,04:00:00,02:00:00,90
7,Phoebe,party,2017-08-04 16:00:00,70,02:00:00,02:00:00,160
8,Phoebe,football,2017-08-04 18:00:00,50,02:00:00,NaT,210


## 5. groupby, max, min for measuring the duration of activities

In section 3 we wanted to know how much time each person spent in each activity. But we overlooked that sometimes we get multiple records for an acitivity which is actually the continuance of the same activities. So to get the actual activity duration we should measure the time from the first consecutive activity appearance to the last. For that we need to mark the change in activities and mark each row with the activity number. We would do this using the .shift command and the .cumsum command we saw before. A new activity is when the activity changes or the person changes.

In [112]:
df['activity_change'] = (df.activity!=df.activity.shift()) | (df.name!=df.name.shift())

Then we will calculate the activity number for each row by grouping per user and applying the glorious .cumsum:

In [113]:
df['activity_num'] = df.groupby('name')['activity_change'].cumsum()
df

,name,activity,timestamp,money_spent,time_diff,row_duration,money_spent_so_far,activity_change,activity_num
3,Harry,party,2017-08-04 07:00:00,20,NaT,NaT,20,True,1.0
2,Jhon,beach,2017-08-04 07:00:00,30,NaT,01:00:00,30,True,1.0
6,Jhon,beach,2017-08-04 08:00:00,60,01:00:00,04:00:00,90,False,1.0
0,Jhon,beach,2017-08-04 12:00:00,50,04:00:00,NaT,140,False,1.0
1,Joey,football,2017-08-04 08:00:00,60,NaT,NaT,60,True,1.0
4,Phoebe,travel,2017-08-04 10:00:00,40,NaT,04:00:00,40,True,1.0
5,Phoebe,football,2017-08-04 14:00:00,50,04:00:00,02:00:00,90,True,2.0
7,Phoebe,party,2017-08-04 16:00:00,70,02:00:00,02:00:00,160,True,3.0
8,Phoebe,football,2017-08-04 18:00:00,50,02:00:00,NaT,210,True,4.0


Now we can calculate the duration of each activity as follows by grouping per name and activity num (and activity — which doesn’t really change the grouping but we need it to have the activity name) and calculating the sum of activity duration per row:

In [123]:
activity_duration = df.groupby(['name','activity_num','activity'])['row_duration'].sum()

In [124]:
df.groupby(['name','activity_num','activity']).head()

,name,activity,timestamp,money_spent,time_diff,row_duration,money_spent_so_far,activity_change,activity_num
3,Harry,party,2017-08-04 07:00:00,20,NaT,NaT,20,True,1.0
2,Jhon,beach,2017-08-04 07:00:00,30,NaT,01:00:00,30,True,1.0
6,Jhon,beach,2017-08-04 08:00:00,60,01:00:00,04:00:00,90,False,1.0
0,Jhon,beach,2017-08-04 12:00:00,50,04:00:00,NaT,140,False,1.0
1,Joey,football,2017-08-04 08:00:00,60,NaT,NaT,60,True,1.0
4,Phoebe,travel,2017-08-04 10:00:00,40,NaT,04:00:00,40,True,1.0
5,Phoebe,football,2017-08-04 14:00:00,50,04:00:00,02:00:00,90,True,2.0
7,Phoebe,party,2017-08-04 16:00:00,70,02:00:00,02:00:00,160,True,3.0
8,Phoebe,football,2017-08-04 18:00:00,50,02:00:00,NaT,210,True,4.0


This will return the activity duration in some kind of timedelta type. You could get the session activity duration in seconds using .dt.total_seconds:

In [125]:
activity_duration = activity_duration.dt.total_seconds()
activity_duration

name    activity_num  activity
Harry   1.0           party           NaN
Jhon    1.0           beach       18000.0
Joey    1.0           football        NaN
Phoebe  1.0           travel      14400.0
        2.0           football     7200.0
        3.0           party        7200.0
        4.0           football        NaN
Name: row_duration, dtype: float64

Then you can the maximal/minimal activity duration for each person (or median or mean) using a command like this:

In [126]:
activity_duration = activity_duration.reset_index().groupby('name').max()

In [127]:
activity_duration

,activity_num,activity,row_duration
name,,,
Harry,1.0,party,NaN
Jhon,1.0,beach,18000.0
Joey,1.0,football,NaN
Phoebe,4.0,travel,14400.0
